In [2]:
!pip install pandas sklearn dask

     |████████████████████████████████| 11.7 MB 1.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 50.9 MB/s eta 0:00:01
     |████████████████████████████████| 26.7 MB 57.2 MB/s eta 0:00:01
     |████████████████████████████████| 701 kB 52.9 MB/s eta 0:00:01
     |████████████████████████████████| 133 kB 65.8 MB/s eta 0:00:01
     |████████████████████████████████| 55 kB 5.3 MB/s  eta 0:00:01
     |████████████████████████████████| 306 kB 54.9 MB/s eta 0:00:01
     |████████████████████████████████| 41.6 MB 57.9 MB/s eta 0:00:01
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1308 sha256=5f527b06e07328eac75e18a3911b01b7bf1725a30fa147c7852f922011cbf189
  Stored in directory: /root/.cache/pip/wheels/22/0b/40/fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897
Successfully built sklearn
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [9]:
!pip install gdown

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 76 kB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 3.1 MB/s eta 0:00:01
  Created wheel for gdown: filename=gdown-4.2.1-py3-none-any.whl size=14429 sha256=6c0ddcc1d92be503eb179cf0fdcca7a7914d54ffa27658f176c3772bcdc8f2bb
  Stored in directory: /root/.cache/pip/wheels/29/0c/4a/58330447f4c918159f819cd72e1fe7d4d2832585a164929f18
Successfully built gdown
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [11]:
!gdown --id 1TkwQqEbkUspNhHz9_OQrad8-QRLm30rt

Downloading...
From: https://drive.google.com/uc?id=1TkwQqEbkUspNhHz9_OQrad8-QRLm30rt
To: /root/transactions_train.csv.zip
100%|████████████████████████████████████████| 613M/613M [00:07<00:00, 85.2MB/s]


In [13]:
!unzip transactions_train.csv.zip

Archive:  transactions_train.csv.zip
  inflating: transactions_train.csv  


In [14]:
!unzip articles.csv.zip

Archive:  articles.csv.zip
  inflating: articles.csv            


In [15]:
!unzip sample_submission.csv.zip

Archive:  sample_submission.csv.zip
  inflating: sample_submission.csv   


In [16]:
!unzip customers.csv.zip

Archive:  customers.csv.zip
  inflating: customers.csv           


In [259]:
import numpy as np
import pandas as pd
from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
import dask.dataframe as dd
from sklearn.model_selection import train_test_split

In [2]:
import tensorflow as tf

In [3]:
def apk(actual, predicted, k=10):
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [4]:
transactions = dd.read_csv(
    'transactions_train.csv',
    # set dtype or pandas will drop the leading '0' and convert to int
    dtype={'article_id': str,
#           't_dat': str,
#            'customer_id':str, 
#           'price':float,
#           'sales_channel_id': 'object'
          } 
)

In [5]:
customers = dd.read_csv('customers.csv', dtype={'customer_id':str, 
#                                                                                                       'FN':int, 
#                                                                                                         'Active':int, 
#                                                                                                       'club_member_status':int,
#                                                                                                         'fashion_news_frequency':str, 
#                                                                                                       'age':float, 
#                                                                                                         'postal_code':str
                                                                                                     })
articles = dd.read_csv('articles.csv', dtype={'article_id':str, 
#                                                                                                     'product_code':int, 'prod_name':str, 
#                                                                                                       'product_type_no':'object',
#                                                                        'product_type_name':str, 'product_group_name':'object', 'graphical_appearance_no':'object',
#                                                                        'graphical_appearance_name':str, 'colour_group_code':'object', 'colour_group_name':str,
#                                                                        'perceived_colour_value_id':str, 'perceived_colour_value_name':str,
#                                                                        'perceived_colour_master_id':str, 'perceived_colour_master_name':str,
#                                                                        'department_no':int, 'department_name':str, 'index_code':str, 'index_name':str,
#                                                                        'index_group_no':'object', 'index_group_name':str, 'section_no':int, 'section_name':str,
#                                                                        'garment_group_no':'object', 'garment_group_name':str, 'detail_desc':str
                                                                                                   })

In [6]:
customers = customers[['customer_id', 'age', 'postal_code']].compute()

In [7]:
mean_age = round(customers.age.dropna().mean())

In [8]:
cust2age = {}
cust2postal = {}
for c, a in customers[['customer_id', 'age']].fillna(mean_age).values:
    cust2age[c] = a

for c, p in customers[['customer_id', 'postal_code']].dropna().values:
    cust2postal[c] = p  


In [9]:
transactions_train = transactions[(transactions['t_dat'] < '2020-09-15') & (transactions['t_dat'] > '2018-09-15')]
transactions_test = transactions[transactions['t_dat'] > '2020-09-15']

In [10]:
transactions_train_arts = dd.merge(transactions_train, articles[['article_id','prod_name']], 
                                   on='article_id', how='left')
transactions_test_arts = dd.merge(transactions_test, articles[['article_id','prod_name']],
                                  on='article_id', how='left')

In [11]:
transactions_train_arts = transactions_train_arts.compute()
transactions_test_arts = transactions_test_arts.compute()

In [112]:
def flatten(lists):
    r = []
    for l in lists:
        for el in l:
            r.append(el)
    return r

def preprocess_dataset(filename, df, cust2age, cust2postal, max_steps=10000):

    f = open(filename, 'w')
    
    grouped = df.groupby('customer_id')
    keys = set(grouped.groups)

    for jj, key in enumerate(keys):
        if jj > max_steps:
            break
        age = cust2age[key]
        postcode = cust2postal[key]
        history_dates_steps = grouped.get_group(key).groupby('t_dat')['prod_name'].agg(list).reset_index()
        history_steps = history_dates_steps['prod_name'].values.tolist()
        dates_steps = history_dates_steps['t_dat'].apply(lambda x: int(x.split('-')[1])).values.tolist()
#         history_steps_articles = grouped.get_group(key).groupby('t_dat')['article_id'].agg(list).values.tolist()[-50:]

        for i in range(len(history_steps)):
#             if np.random.choice([1,0]):
#                 continue
            current_history = '##'.join(flatten(history_steps[:i])).replace('\t', ' ')
            f.write('\t'.join([str(round(age)), 
                               postcode, 
                               current_history, 
                               '##'.join(history_steps[i]),
                               str(dates_steps[i]),
                               key]) + '\n')

    f.close()

def preprocess_for_inference(filename, df, cust2age, cust2postal):
    f = open(filename, 'w')
    

    for key, target, history, dat in df[['customer_id', 'prod_name', 'prod_name_history', 't_dat']].fillna(' ').values:
        age = cust2age[key]
        postcode = cust2postal[key]

        history = '##'.join(history.replace('\t', ' ').split('##'))
        f.write('\t'.join([str(round(age)), postcode, history, target, str(dat), key]) + '\n')

    f.close()

def preprocess_for_submission(filename, df, cust2age, cust2postal):
    f = open(filename, 'w')
    

    for key, history in df[['customer_id', 'prod_name']].fillna(' ').values:
        age = cust2age[key]
        postcode = cust2postal[key]

        history = '##'.join(history.replace('\t', ' ').split('##')[-50:])
        f.write('\t'.join([str(round(age)), postcode, history, ' ', '9', key]) + '\n')

    f.close()

In [261]:
# grouped = transactions_test_arts.groupby('customer_id')

In [260]:
# grouped['t_dat'].agg('nunique').sort_values()

In [57]:
# del grouped

In [58]:
# groups = set(grouped.groups)

In [ ]:
# groups

In [99]:
# dates = grouped.get_group('76708d589a34d051205e42ce8db8da20aeb5f3954c395da7b76bb0db11fc648b') \
#                             .groupby('t_dat')['prod_name'] \
#                             .agg(list).reset_index()['t_dat']

In [262]:
# dates

In [250]:
transactions_train_arts.reset_index(inplace=True, drop=True)

In [255]:
train_df_cid, valid_df_cid = train_test_split(transactions_train_arts.customer_id.unique())

In [257]:
preprocess_dataset('train_dataset.csv', 
                   transactions_train_arts[transactions_train_arts.customer_id.isin(train_df_cid)], cust2age, 
                   cust2postal, max_steps=100000)


In [258]:
preprocess_dataset('test_dataset.csv', 
                   transactions_train_arts[transactions_train_arts.customer_id.isin(valid_df_cid)], 
                   cust2age, cust2postal)

In [116]:
infer_df = transactions_test_arts.groupby('customer_id')[['prod_name', 't_dat']] \
.agg({'prod_name':'##'.join, 't_dat':lambda x: list(x)[0]}) \
.join(
    transactions_train_arts.groupby('customer_id')['prod_name'].agg('##'.join), how='left', rsuffix='_history'
                                                                                 ).reset_index()

In [117]:
infer_df['t_dat'] = infer_df['t_dat'].apply(lambda x: int(x.split('-')[1]))

In [118]:
preprocess_for_inference('infer_df_valid_dataset.csv', infer_df, cust2age, cust2postal)

In [18]:
# submussion_df = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')

In [19]:
# submision_df = submussion_df.join(transactions_train_arts.groupby('customer_id')['prod_name'].agg('##'.join), on='customer_id', how='left', 
                                                                                 )

In [20]:
# preprocess_for_submission('submission_dataset.csv', submision_df, cust2age, cust2postal)

In [263]:
# dataset = pd.read_csv('train_dataset.csv', sep='\t', header=None, names=['age', 'post', 'history', 'target'])
dataset_test = pd.read_csv('test_dataset.csv', sep='\t', header=None, names=['age', 'post', 'history', 'target', 
                                                                             't_dat','customer_id'])
valid_dataset = pd.read_csv('infer_df_valid_dataset.csv', sep='\t', header=None, names=['age', 'post', 'history', 
                                                                                        'target', 't_dat', 'customer_id'])

In [264]:
dataset_test

,age,post,history,target,t_dat,customer_id
0,22,2e477694b63ada8920e647fe222780b22c8d8c0788c3a8...,NaN,Chestnut strap top##Dame##Rodin##TILLY BLOUSE#...,4,7af3b232b25d4b7960e4b817aa756bf10b93e6e513c264...
1,52,d0ecb24786dac0cdcc957cafbb5bd5151793eb27f4e106...,NaN,TP THEO hoodie BB##TP THEO hoodie BB,10,491ff1894b96bfa2e827519917d413fe44fb06e7446c8e...
2,52,d0ecb24786dac0cdcc957cafbb5bd5151793eb27f4e106...,TP THEO hoodie BB##TP THEO hoodie BB,Mister Muscle LS##CA Gustavsberg TVP,4,491ff1894b96bfa2e827519917d413fe44fb06e7446c8e...
3,42,24b5423a918bd5269176fe85e311554a4df5d838c16c3e...,NaN,Tiblisi paperwaist tapered##KELLY SHIRT##KELLY...,5,53262b1e8d0bc2a29f5e92915de90c64129ec9ae06fb83...
4,25,6c63be0de195adafd7065873c88fdf78081191aa529fe7...,NaN,ED Madison Skinny HW##Skinny RW denim##ED Desp...,10,fb160bc5168dd942886289b772987a28ca28ad687e1f92...
...,...,...,...,...,...,...
65692,50,0d3ef8cd70bd193de99456e9b50984f10268ae1e5edc87...,W YODA KNIT OL OFFER##Pablo coat##Angel Hoodie...,Wake rib polo,12,b0fab595b692dfe7bb2d2df0fe600ad862dc9c3d7c8b9c...
65693,17,33d40124ebc670caa116f1b2395a31f799c1835205eb87...,NaN,Tilda tank##Tilda tank##Ghost LS,3,ffe6d70abd5cbb9ace0630da24f6224b4925f3fd6b90b9...
65694,21,f94a369132c077f64448855fe9bac7dc4c9c2f4d79c8d9...,NaN,Pablo sweatshirt,12,c406e0ff833aaf745bf565970a81d1e779c8fabfdac380...
65695,21,f94a369132c077f64448855fe9bac7dc4c9c2f4d79c8d9...,Pablo sweatshirt,Hazelnut Push Melbourne##Pilar high support,5,c406e0ff833aaf745bf565970a81d1e779c8fabfdac380...


In [19]:
# dataset_test.groupby('customer_id')['target'].agg(' '.join)

In [265]:
from collections import Counter
prod_counts = Counter()
post_counts = Counter()

for line in open('train_dataset.csv'):
    _, post, _, target, d, customer_id = line.strip('\n').split('\t')
    prod_counts.update(target.split('##'))
    post_counts[post] += 1 

In [271]:
prod_counts.most_common()

[('Jade HW Skinny Denim TRS', 12116),
 ('Luna skinny RW', 10748),
 ('Timeless Midrise Brief', 9145),
 ('Tilly (1)', 7715),
 ('Cat Tee.', 5901),
 ('Simple as That Triangle Top', 5554),
 ('Shake it in Balconette', 5393),
 ('Despacito', 5047),
 ('Tilda tank', 4949),
 ('Simple as that Cheeky Tanga', 4903),
 ('Nora T-shirt', 4760),
 ('Skinny Ankle R.W Brooklyn', 4723),
 ('Pluto RW slacks (1)', 4480),
 ('SUPREME RW tights', 4332),
 ('Primo slacks', 4283),
 ('Madison skinny HW (1)', 4121),
 ('7p Basic Shaftless', 4101),
 ('Julia RW Skinny Denim TRS', 3914),
 ('Hazelnut Push Melbourne', 3636),
 ('Brit Baby Tee', 3563),
 ('Kanta slacks RW', 3493),
 ('Therese tee', 3482),
 ('Melrose', 3325),
 ('Lazer Razer Brief', 3289),
 ('Perrie Slim Mom Denim TRS', 3253),
 ('New Girl Push Top', 3174),
 ('RICHIE HOOD', 3169),
 ('SUPREME tights', 3165),
 ('Becka hoodie', 3126),
 ('Charlie Top', 3090),
 ('Mariette Blazer', 3084),
 ('Shaping Skinny H.W', 2993),
 ('Moa tank', 2971),
 ('Lucy blouse', 2912),
 ('Push

In [266]:
prod2id = {'PAD':0, 'UNK':1}
for k in prod_counts:
    if prod_counts[k] > 10:
        prod2id[k] = len(prod2id)
id2prod = {i:k for k, i in prod2id.items()}

post2id = {'UNK':0}
for k in post_counts:
    if post_counts[k] > 10:
        post2id[k] = len(post2id)
id2post = {i:k for k, i in post2id.items()}

In [146]:
# articles.loc[100:120, ['prod_name', 'detail_desc', 'article_id']].compute().values

In [267]:
target_size = len(id2prod)

In [268]:
target_size

19923

In [25]:
# def generate(filename, infer=False):
#     for line in open(filename):
#         age, post, history, target, _ = line.strip('\n').split('\t') 
#         target = Counter([prod2id.get(p, 0) for p in target.split('##') if p in prod2id])
#         s = sum(target.values())
# #         target = list(set([prod2id.get(p, 0) for p in target.split('##') if p in prod2id]))
# #         if skip_empty and not target:
# #             continue
#         post = post2id.get(post, 0)
        
        
#         if not infer:
#             target_tensor = np.zeros((target_size))
#             for t in target:
#                 target_tensor[t] = t#/s
#             yield ((tf.constant([float(age)], dtype=tf.float64), tf.constant([post], dtype=tf.int32), tf.constant([history])), tf.constant(target_tensor, dtype=tf.float64))
#         else:
#             yield (tf.constant([float(age)], dtype=tf.float64), tf.constant([post], dtype=tf.int32), tf.constant([history]))

In [272]:
def generate(filename, infer=False):
    for line in open(filename):
        age, post, history, target, dat, _ = line.strip('\n').split('\t') 
        target = Counter([prod2id.get(p, 1) for p in target.split('##') if p in prod2id])
        history = [prod2id.get(p, 1) for p in history.split('##') if p in prod2id]
        h_array = np.zeros(50)
        if history:
            h_array[-min(50, len(history)):] = history[:50]
#         s = sum(target.values())
#         target = list(set([prod2id.get(p, 0) for p in target.split('##') if p in prod2id]))
#         if skip_empty and not target:
#             continue
        post = post2id.get(post, 0)
        
        
        if not infer:
            target_tensor = np.zeros((target_size))
            for t in target:
                target_tensor[t] = t#/s
            yield ((tf.constant([float(age)], dtype=tf.float64), 
                    tf.constant([post], dtype=tf.int32), 
                    tf.constant(h_array, dtype=tf.int32),
                    tf.constant([int(dat)-1], dtype=tf.int32)), 
                   
                   tf.constant(target_tensor, dtype=tf.float64))
        else:
            yield (tf.constant([float(age)], dtype=tf.float64),
                   tf.constant([post], dtype=tf.int32), 
                   tf.constant(h_array, dtype=tf.int32),
                   tf.constant([int(dat)-1], dtype=tf.int32))

In [273]:
# a = open('infer_df_valid_dataset.csv')

In [274]:
dataset = tf.data.Dataset.from_generator(generate, 
                                         output_signature=((tf.TensorSpec(shape=(1,), dtype=tf.float64),
                                                            tf.TensorSpec(shape=(1,), dtype=tf.int32),
                                                            tf.TensorSpec(shape=(50,), dtype=tf.int32),
                                                            tf.TensorSpec(shape=(1,), dtype=tf.int32)),
                                                            tf.TensorSpec(shape=(target_size), dtype=tf.float64)
                                                           ),
                                         args=['train_dataset.csv']).batch(128).prefetch(5)

dataset_test = tf.data.Dataset.from_generator(generate, 
                                         output_signature=((tf.TensorSpec(shape=(1,), dtype=tf.float64),
                                                            tf.TensorSpec(shape=(1,), dtype=tf.int32),
                                                            tf.TensorSpec(shape=(50,), dtype=tf.int32),
                                                            tf.TensorSpec(shape=(1,), dtype=tf.int32)),
                                                            tf.TensorSpec(shape=(target_size), dtype=tf.float64)
                                                           ),
                                         args=['test_dataset.csv']).batch(128)

dataset_valid = tf.data.Dataset.from_generator(generate, 
                                         output_signature=(tf.TensorSpec(shape=(1,), dtype=tf.float64),
                                                            tf.TensorSpec(shape=(1,), dtype=tf.int32),
                                                            tf.TensorSpec(shape=(50,), dtype=tf.int32),
                                                          tf.TensorSpec(shape=(1,), dtype=tf.int32)),
                                         args=['infer_df_valid_dataset.csv', True]).batch(200)

# dataset_submission = tf.data.Dataset.from_generator(generate, 
#                                          output_signature=(tf.TensorSpec(shape=(1,), dtype=tf.float64),
#                                                             tf.TensorSpec(shape=(1,), dtype=tf.int32),
#                                                             tf.TensorSpec(shape=(1,), dtype=tf.string)),
#                                          args=['submission_dataset.csv', True]).batch(300)

In [275]:
# dataset_submission = tf.data.Dataset.from_generator(generate, 
#                                          output_signature=(tf.TensorSpec(shape=(1,), dtype=tf.float64),
#                                                             tf.TensorSpec(shape=(1,), dtype=tf.int32),
#                                                             tf.TensorSpec(shape=(1,), dtype=tf.string)),
#                                          args=['submission_dataset.csv', True]).batch(1000).prefetch(100)

In [276]:
# texts = dataset.map(lambda x, y: x[2])

In [277]:
# from tensorflow.keras.layers import TextVectorization
# import string
# import re


# def custom_standardization(input_data):
#     lowercase = tf.strings.lower(input_data)
#     text = tf.strings.regex_replace(lowercase, "##", " #SEP# ")
#     return text


# max_features = 50000
# sequence_length = 100

# vectorize_layer = TextVectorization(
#     standardize=custom_standardization,
#     max_tokens=max_features,
#     output_mode="int",
#     output_sequence_length=sequence_length
# )





In [278]:
# vectorize_layer.adapt(texts)

In [156]:
g = generate('train_dataset.csv')

In [159]:
next(g)

((<tf.Tensor: shape=(1,), dtype=float64, numpy=array([37.])>,
  <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0], dtype=int32)>,
  <tf.Tensor: shape=(50,), dtype=int32, numpy=
  array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3,
         4, 2, 5, 6, 7, 8], dtype=int32)>,
  <tf.Tensor: shape=(1,), dtype=int32, numpy=array([7], dtype=int32)>),
 <tf.Tensor: shape=(19898,), dtype=float64, numpy=array([0., 0., 0., ..., 0., 0., 0.])>)

In [47]:
# import tensorflow_addons as tfa

In [279]:
inputs_age = tf.keras.layers.Input(shape=(1,), dtype=tf.float64)
inputs_postal = tf.keras.layers.Input(shape=(1,), dtype=tf.int32)
inputs_history = tf.keras.layers.Input(shape=(50,), dtype=tf.int32)
inputs_season = tf.keras.layers.Input(shape=(1,), dtype=tf.int32)

# history = vectorize_layer(inputs_history)

embeddings_postal = tf.keras.layers.Embedding(input_dim=len(post2id), output_dim=20)(inputs_postal )
embeddings_season = tf.keras.layers.Embedding(input_dim=12, output_dim=5)(inputs_season )
embeddings_history = tf.keras.layers.Embedding(input_dim=len(prod2id), output_dim=100)(inputs_history)

dense_age = tf.keras.layers.Dense(10, activation='relu')(inputs_age)
dense_age_flatten = tf.keras.layers.Flatten()(dense_age)

dense_post = tf.keras.layers.Dense(10, activation='relu')(embeddings_postal)
dense_post_flatten = tf.keras.layers.Flatten()(dense_post)

dense_season = tf.keras.layers.Dense(10, activation='relu')(embeddings_season)
dense_season_flatten = tf.keras.layers.Flatten()(dense_season)




# mean = tf.keras.layers.Lambda(lambda x: tf.keras.backend.mean(x,  axis=1))(embeddings_history)
lstm_history = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(512, return_sequences=False))(embeddings_history)
# conv1 = tf.keras.layers.Conv1D(kernel_size=7, filters=128, strides=1, activation='relu')(embeddings_history)
# conv2 = tf.keras.layers.Conv1D(kernel_size=5, filters=128, strides=1, activation='relu')(conv1)
# conv3 = tf.keras.layers.Conv1D(kernel_size=3, filters=128, strides=1, activation='relu')(conv2)
# pool = tf.keras.layers.AveragePooling1D()(conv3)
# flat = tf.keras.layers.Flatten()(pool)


# convs = []

# for ks in [3,4,5,7,10,20]:
#     conv1 = tf.keras.layers.Conv1D(kernel_size=ks, filters=256, padding='same', activation='relu',
#                                    strides=1)(embeddings_history)
#     conv2 = tf.keras.layers.Conv1D(kernel_size=ks, filters=256, padding='same',strides=1, 
#                                   activation='relu')(conv1)
#     convs.append(conv2)


# concat_convs = tf.keras.layers.concatenate(convs, axis=2)

# conv_global = tf.keras.layers.Conv1D(kernel_size=5, filters=256, strides=1, activation='relu')(concat_convs)
# flatten_conv = tf.keras.layers.Flatten()(conv_global)


dense_general = tf.keras.layers.Dense(100, activation='relu')(lstm_history)

concat = tf.keras.layers.concatenate([
                                      dense_age_flatten,
                                      dense_post_flatten, 
                                      dense_general,
                                      dense_season_flatten,
#                                       flatten_conv
#                                       flat
                                     ], axis=-1)


output = tf.keras.layers.Dense(target_size, activation='relu')(concat)


model = tf.keras.Model(inputs=[inputs_age, inputs_postal, inputs_history, inputs_season], 
                       outputs=output)
optimizer = tf.keras.optimizers.Adam()


model.compile(optimizer=optimizer,
                  loss=tf.keras.losses.CosineSimilarity(),
#               loss=tfa.losses.()
              )

In [280]:
print(model.summary())

Model: "model_25"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_113 (InputLayer)         [(None, 1)]          0           []                               
                                                                                                  
 input_114 (InputLayer)         [(None, 50)]         0           []                               
                                                                                                  
 input_115 (InputLayer)         [(None, 1)]          0           []                               
                                                                                                  
 input_112 (InputLayer)         [(None, 1)]          0           []                               
                                                                                           

In [281]:
model.fit(dataset, validation_data=dataset_test, epochs=10, verbose=1)

Epoch 1/10
5148/5148 [==============================] - 564s 109ms/step - loss: -0.0098 - val_loss: -0.0105
Epoch 2/10
5148/5148 [==============================] - 561s 109ms/step - loss: -0.0112 - val_loss: -0.0113
Epoch 3/10
5148/5148 [==============================] - ETA: 0s - loss: -0.0134

KeyboardInterrupt: 

In [184]:
model.evaluate(dataset_test)

88/88 [==============================] - 9s 98ms/step - loss: -0.0081


-0.00806447770446539

In [231]:
pred_articles = []
for batch in dataset_valid:
    preds = model.predict_on_batch(batch)
    top_12 = preds.argsort(1)[:, :-13:-1]
    
    for i in range(preds.shape[0]):
        filtered_top_12 = '##'.join([id2prod[p] for p in top_12[i] if preds[i][p] > 0.05])
        pred_articles.append(filtered_top_12)

In [232]:
preds

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [233]:
valid_dataset['prediction'] = pred_articles

In [234]:
valid_dataset

,age,post,history,target,t_dat,customer_id,prediction
0,27,43cbf97df3d118b937551fb21a08d513bfb2e58223315f...,Jennifer (1)##Cat Tee##Pamela Tee.##Kirsten##M...,Brit Baby Tee,9,00039306476aaf41a07fed942884f16b30abfa83a2a8be...,Pluto RW slacks (1)##Milk RW slack##Jade HW Sk...
1,33,d647e4ede3d0eb4ce0750440a110350b5f4c758165d89d...,Blake Sandal##Svea Cropped Tank##Connie Cut Ou...,DONNA DENIM SKIRT,9,0003e867a930d0d6842f923d6ba7c9b77aba33fe2a0fbf...,Jade HW Skinny Denim TRS##RICHIE HOOD##Timeles...
2,29,72afbb92c0200628bfa8f983c241eb0dc14e107f87d95b...,H2 Carnation solid##Bowie jumpsuit##Skinny H.W...,Skinny H.W Ankle Festive##Maja cargo Slim HW D...,9,000493dd9fc463df1acc2081450c9e75ef8e87d5dd17ed...,Jade HW Skinny Denim TRS##Superwoman Superpush...
3,25,ed323346483de9f9b9ac7d73d34e0c87b5946d09da3b07...,,Herbal Hood -TVP,9,000525e3fe01600d717da8423643a8303390a055c578ed...,Jade HW Skinny Denim TRS##Curvy Jeggings HW An...
4,23,3119ea10ffe5ac3419b9127589a61b33e1ae38ecbb997b...,Bobo##W YODA KNIT OL OFFER##Jafar##Peggy Trous...,Mini me Tassle fur##Chilly padded coat##Highwa...,9,00077dbd5c4a4991e092e63893ccf29294a9d5c46e8501...,Antonia heavy t-shirt##Jen tee##Lemon HW##Bail...
...,...,...,...,...,...,...,...
68979,24,ef06cc9c740983fed082a8cc961cbcb4db1b40c8a2a369...,Twelve top##Nord tee##Frida Printed Sweater##D...,Barrista cardigan##Bama1,9,fffa67737587e52ff1afa9c7c6490b5eb7acbc439fe82b...,Jade HW Skinny Denim TRS##Jen tee##Milk RW sla...
68980,50,ce29e6932c6263c659e14c0f170fd8d2c332fbbae00144...,Heart LS##W YODA KNIT OL OFFER##Scallop 5p Soc...,Sunspot Seamless Crop Top##Acai seamless bra C...,9,fffa7d7799eb390a76308454cbdd76e473d65b1497fbe4...,Timeless Midrise Brief##RICHIE HOOD##Shake it ...
68981,25,102183de22c6110db8e6e9b520a7aaab65c0cee1c5ff5a...,Curvy Jeggings HW Ankle##Michaela wallet big##...,Glamping##Kelso##Becka hoodie##Kelso##ED Barab...,9,fffae8eb3a282d8c43c77dd2ca0621703b71e90904dfde...,Jade HW Skinny Denim TRS##Bowie##Push Up Jeggi...
68982,45,ed6e0192fc1235342b0211e38e95d81d146fa09b7e0e47...,SKINNY Knit 89##Skinny Biker Trash##JOGGER Kni...,DeLuca pull on TVP RW##ESSENTIAL YOKO ROLLER NECK,9,fffd870c6324ad3bda24e4d6aeae221c199479086bfdfd...,Jade HW Skinny Denim TRS##Timeless Midrise Bri...


In [235]:
mapk(valid_dataset['target'].str.split("##"), valid_dataset['prediction'].str.split('##'), 12)

0.010450144905261393

In [ ]:
pred_articles_sub = []
for ij, batch in enumerate(dataset_submission):
    if not (ij+1) % 100:
        print(ij * 500)
    preds = model.predict_on_batch(batch)
    top_12 = preds.argsort(1)[:, :-13:-1]
    
    for i in range(preds.shape[0]):
        filtered_top_12 = ' '.join([id2prod[p] for p in top_12[i] if preds[i][p] > 0.30])
        pred_articles_sub.append(filtered_top_12)

In [ ]:
submision_df['prediction'] = pred_articles_sub

In [ ]:
submision_df

In [ ]:
submision_df[['customer_id', 'prediction']].to_csv("submission.csv", index=False, header=True)